# Import Libraries

In [20]:
import numpy as np
import matplotlib.pyplot as plt

import GPy
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

from art.attacks.evasion import HighConfidenceLowUncertainty
from art.estimators.classification import GPyGaussianProcessClassifier

# Load Data

In [11]:
breast_cancer = load_breast_cancer()

In [12]:
X, y = breast_cancer.data, breast_cancer.target

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
scaler = StandardScaler()

In [15]:
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model

In [16]:
kernel = GPy.kern.RBF(input_dim=X_train.shape[1])

In [17]:
model = GPy.models.GPClassification(X_train, y_train.reshape(-1, 1), kernel=kernel)

In [18]:
model.optimize()

In [19]:
y_pred = model.predict(X_test)[0]

In [23]:
print("Accuracy:", accuracy_score(y_test, y_pred.round()) * 100)

Accuracy: 97.36842105263158


In [24]:
classifier = GPyGaussianProcessClassifier(model)

# Attack

In [25]:
attack1 = HighConfidenceLowUncertainty(classifier, conf=0.8, min_val=-1.0, max_val=2.0)

In [26]:
adv_confidence = attack1.generate(X_test)

HCLU:   0%|          | 0/114 [00:00<?, ?it/s]

 /home/alper/.local/lib/python3.10/site-packages/scipy/optimize/_optimize.py:404: RuntimeWarning:Values in x were outside bounds during a minimize step, clipping to bounds


In [34]:
attack2 = HighConfidenceLowUncertainty(classifier, unc_increase=0.9, min_val=0.0, max_val=2.0)

In [35]:
adv_uncertainty = attack2.generate(X_test)

HCLU:   0%|          | 0/114 [00:00<?, ?it/s]

# Results

In [33]:
print("Accuracy (High Confidence):", accuracy_score(y_test, np.argmax(adv_confidence, axis=1)) * 100)

Accuracy: 8.771929824561402


In [36]:
print("Accuracy (Low Uncertainty):", accuracy_score(y_test, np.argmax(adv_uncertainty, axis=1)) * 100)

Accuracy (Low Uncertainty): 7.017543859649122
